tgb - 11/13/2019 - Testing if rescaling the inputs changes the ability of the network to generalize.   
Architecture = From [], predict []   
Test 1 = Rephrase all water concentrations as RH  
Test 2 = Rephrase all temperature as deviations from moist adiabat  
  
tgb - 11/15/2019 - This notebook is mostly used for pre-processing

# 0) Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
import tensorflow as tf
from tensorflow import math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
# physical_devices = tf.config.experimental.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.config.experimental.set_memory_growth(physical_devices[1], True)
# tf.config.experimental.set_memory_growth(physical_devices[2], True)
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM



/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog
[Errno 2] No such file or directory: '/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM'
/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


# 1) Preprocessing

In [2]:
%cd /export/nfs0home/tbeucler/CBRAIN-CAM

/nfspool-0/home/tbeucler/CBRAIN-CAM


In [6]:
!python preprocessing-11132019.py -c pp_config/2021_09_02_Small_data_test_For_Nando.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
09/02/2021 03:53:58 AM Preprocess training dataset with real_geography flag set to False
09/02/2021 03:53:58 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_small.nc
09/02/2021 03:53:58 AM Reading input files
09/02/2021 03:53:58 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[5-6]-01-00000.nc
09/02/2021 03:54:02 AM Crop levels
09/02/2021 03:54:02 AM Create stacked dataarray
09/02/2021 03:54:02 AM Real geography flag set to False
09/02/2021 03:54:02 AM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
var is  RH
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set r

OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166890 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166955 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166956 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166957 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166958 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166959 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166960 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166961 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166962 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166964 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166965 thread 59 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166963 thread 

OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166875 thread 144 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167052 thread 145 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167053 thread 146 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167054 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167055 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167056 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167057 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167058 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167060 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167059 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167061 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167

OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166881 thread 240 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167145 thread 241 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167146 thread 242 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167147 thread 243 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167149 thread 245 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167148 thread 244 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167150 thread 246 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167151 thread 247 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167152 thread 248 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167153 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167154 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167

OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166889 thread 336 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167282 thread 338 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167281 thread 337 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167283 thread 339 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167284 thread 340 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167285 thread 341 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167287 thread 343 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167286 thread 342 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167288 thread 344 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167289 thread 345 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167291 thread 347 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167

OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166853 thread 432 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167380 thread 434 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167381 thread 435 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167379 thread 433 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167382 thread 436 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167383 thread 437 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167384 thread 438 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167385 thread 439 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167386 thread 440 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167389 thread 443 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167388 thread 442 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167

OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166849 thread 528 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167476 thread 529 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167477 thread 530 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167478 thread 531 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167479 thread 532 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167480 thread 533 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167481 thread 534 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167483 thread 536 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167482 thread 535 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167485 thread 538 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167486 thread 539 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167

OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166867 thread 624 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167574 thread 625 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167575 thread 626 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167576 thread 627 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167578 thread 629 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167577 thread 628 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167580 thread 631 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167579 thread 630 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167581 thread 632 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167582 thread 633 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167583 thread 634 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167

OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166852 thread 719 bound to OS proc set 39
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167671 thread 720 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167672 thread 721 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167673 thread 722 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167674 thread 723 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167675 thread 724 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167676 thread 725 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167678 thread 727 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167677 thread 726 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167679 thread 728 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167680 thread 729 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167

OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167828 thread 834 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166862 thread 835 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167829 thread 836 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 166863 thread 837 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167830 thread 838 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167831 thread 839 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167832 thread 840 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167834 thread 842 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167835 thread 843 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167836 thread 844 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 166763 tid 167833 thread 841 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 166763 ti

In [5]:
!python preprocessing-11132019.py -c pp_config/2021_09_02_Train_Valid_For_Nando.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
09/02/2021 01:01:04 PM Preprocess training dataset with real_geography flag set to False
09/02/2021 01:01:04 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_For_Nando.nc
09/02/2021 01:01:04 PM Reading input files
09/02/2021 01:01:04 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-4]-*-*.nc
09/02/2021 01:01:16 PM Crop levels
09/02/2021 01:01:16 PM Create stacked dataarray
09/02/2021 01:01:16 PM Real geography flag set to False
09/02/2021 01:01:16 PM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
These time steps are cut: []
09/02/2021 01:01:16 PM Stack and reshape dataarray
09/02/2021 01:01:49 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TRAIN_Fo

OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158169 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158267 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158266 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158269 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158268 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158270 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158271 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158272 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158273 thread 59 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158274 thread 60 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158275 thread 61 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158276 thread

OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158181 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158363 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158364 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158365 thread 156 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158367 thread 158 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158366 thread 157 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158369 thread 160 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158368 thread 159 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158370 thread 161 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158371 thread 162 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158372 thread 163 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158

OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158193 thread 255 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158459 thread 256 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158460 thread 257 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158461 thread 258 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158462 thread 259 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158463 thread 260 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158464 thread 261 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158465 thread 262 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158466 thread 263 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158468 thread 265 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158467 thread 264 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 

OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158205 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158560 thread 358 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158561 thread 359 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158562 thread 360 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158563 thread 361 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158565 thread 363 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158564 thread 362 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158567 thread 365 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158566 thread 364 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158568 thread 366 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158569 thread 367 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 157429 ti

OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158170 thread 456 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158655 thread 457 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158656 thread 458 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158657 thread 459 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158658 thread 460 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158659 thread 461 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158660 thread 462 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158661 thread 463 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158662 thread 464 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158663 thread 465 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158664 thread 466 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 157429 ti

OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158188 thread 555 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158754 thread 556 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158755 thread 557 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158189 thread 558 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158757 thread 560 bound to OS proc set 44
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158756 thread 559 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158758 thread 561 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158760 thread 563 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158761 thread 564 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158762 thread 565 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158763 thread 566 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 157429 ti

OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158206 thread 657 bound to OS proc set 46
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158851 thread 658 bound to OS proc set 42
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158852 thread 659 bound to OS proc set 38
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158207 thread 660 bound to OS proc set 25
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158853 thread 661 bound to OS proc set 29
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158855 thread 663 bound to OS proc set 35
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158854 thread 662 bound to OS proc set 33
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158857 thread 665 bound to OS proc set 27
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158858 thread 666 bound to OS proc set 37
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158859 thread 667 bound to OS proc set 41
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 158861 thread 669 bound to OS proc set 47
OMP: Info #250: KMP_AFFINITY: pid 157429 ti

OMP: Info #250: KMP_AFFINITY: pid 157429 tid 159112 thread 771 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 159113 thread 772 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 159114 thread 773 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 159115 thread 774 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 159117 thread 776 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 159116 thread 775 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 159118 thread 777 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 159119 thread 778 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 159120 thread 779 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 159121 thread 780 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 159122 thread 781 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 157429 tid 15

In [6]:
!python preprocessing-11132019.py -c pp_config/2021_09_02_Test_for_Nando.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
09/02/2021 01:20:54 PM Preprocess training dataset with real_geography flag set to False
09/02/2021 01:20:54 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TEST_For_Nando.nc
09/02/2021 01:20:54 PM Reading input files
09/02/2021 01:20:54 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-0[6-9]-*-*.nc
09/02/2021 01:22:16 PM Crop levels
09/02/2021 01:22:16 PM Create stacked dataarray
09/02/2021 01:22:16 PM Real geography flag set to False
09/02/2021 01:22:16 PM Aquaplanet simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  FSNT
var is  FSNS
var is  FLNT
var is  FLNS
var is  PRECT
These time steps are cut: []
09/02/2021 01:22:16 PM Stack and reshape dataarray
09/02/2021 01:22:49 PM Save dataarray as /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_02_TEST_For_

OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169021 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169125 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169124 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169023 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169130 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169131 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169132 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169133 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169135 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169134 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169137 thread 59 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169136 thread 

OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169033 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169229 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169230 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169035 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169233 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169232 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169235 thread 157 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169234 thread 156 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169236 thread 158 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169237 thread 159 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169239 thread 161 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 16

OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169045 thread 252 bound to OS proc set 1
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169338 thread 253 bound to OS proc set 5
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169339 thread 254 bound to OS proc set 9
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169047 thread 255 bound to OS proc set 11
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169381 thread 256 bound to OS proc set 7
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169383 thread 258 bound to OS proc set 13
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169384 thread 259 bound to OS proc set 17
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169382 thread 257 bound to OS proc set 3
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169385 thread 260 bound to OS proc set 21
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169386 thread 261 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169387 thread 262 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169

OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169059 thread 357 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169501 thread 358 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169502 thread 359 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169503 thread 360 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169504 thread 361 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169505 thread 362 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169506 thread 363 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169507 thread 364 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169508 thread 365 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169509 thread 366 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169510 thread 367 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 167117 ti

OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169855 thread 453 bound to OS proc set 23
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169856 thread 454 bound to OS proc set 19
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169859 thread 457 bound to OS proc set 28
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169857 thread 455 bound to OS proc set 15
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169861 thread 459 bound to OS proc set 34
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169862 thread 460 bound to OS proc set 30
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169863 thread 461 bound to OS proc set 26
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169860 thread 458 bound to OS proc set 32
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169858 thread 456 bound to OS proc set 24
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169864 thread 462 bound to OS proc set 36
OMP: Info #250: KMP_AFFINITY: pid 167117 tid 169865 thread 463 bound to OS proc set 40
OMP: Info #250: KMP_AFFINITY: pid 167117 ti

In [9]:
!python preprocessing-11132019.py -c pp_config/2021_09_03_Valid_for_Nando_Cl_Inv.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
09/03/2021 12:18:58 AM Preprocess training dataset with real_geography flag set to False
09/03/2021 12:18:58 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_03_VALID_For_Nando_ClInv.nc
09/03/2021 12:18:58 AM Reading input files
09/03/2021 12:18:58 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[5-8]-*-*.nc
09/03/2021 12:22:01 AM Crop levels
09/03/2021 12:22:01 AM Create stacked dataarray
09/03/2021 12:22:01 AM Real geography flag set to False
09/03/2021 12:22:01 AM Aquaplanet simulation detected
var is  RH
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-47
OMP: Info #156: KMP_AFFINITY: 48 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 2 packages x 12 c

OMP: Info #250: KMP_AFFINITY: pid 198864 tid 203780 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204817 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204818 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204819 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204820 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204821 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204822 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204823 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204824 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204826 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204825 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204827 thread 

OMP: Info #250: KMP_AFFINITY: pid 198864 tid 203811 thread 144 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204920 thread 145 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204921 thread 146 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204922 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204923 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204924 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204925 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204926 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204927 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204928 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204930 thread 155 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 204

OMP: Info #250: KMP_AFFINITY: pid 198864 tid 203779 thread 240 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205018 thread 241 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205019 thread 242 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205020 thread 243 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205022 thread 245 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205021 thread 244 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205023 thread 246 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205025 thread 248 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205027 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205024 thread 247 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205028 thread 251 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205

OMP: Info #250: KMP_AFFINITY: pid 198864 tid 203804 thread 336 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205161 thread 337 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205162 thread 338 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205163 thread 339 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205164 thread 340 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205165 thread 341 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205166 thread 342 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205167 thread 343 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205168 thread 344 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205169 thread 345 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205170 thread 346 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 205

...

OMP: Info #250: KMP_AFFINITY: pid 198864 tid 203789 thread 720 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206170 thread 721 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206171 thread 722 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206172 thread 723 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206173 thread 724 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206174 thread 725 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206176 thread 727 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206175 thread 726 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206177 thread 728 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206178 thread 729 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206179 thread 730 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206

OMP: Info #250: KMP_AFFINITY: pid 198864 tid 203766 thread 816 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206321 thread 818 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206320 thread 817 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206323 thread 820 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206322 thread 819 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206324 thread 821 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206325 thread 822 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206327 thread 824 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206326 thread 823 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206328 thread 825 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206329 thread 826 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206

OMP: Info #250: KMP_AFFINITY: pid 198864 tid 203797 thread 912 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206742 thread 913 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206743 thread 914 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206745 thread 915 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206746 thread 916 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206747 thread 917 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206748 thread 918 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206749 thread 919 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206750 thread 920 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206751 thread 921 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206752 thread 922 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206

OMP: Info #250: KMP_AFFINITY: pid 198864 tid 203774 thread 1008 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206983 thread 1009 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206984 thread 1010 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206985 thread 1011 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206986 thread 1012 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206987 thread 1013 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206988 thread 1014 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206990 thread 1016 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206989 thread 1015 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206991 thread 1017 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 206992 thread 1018 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 198

OMP: Info #250: KMP_AFFINITY: pid 198864 tid 203781 thread 1104 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 207143 thread 1105 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 207144 thread 1106 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 207145 thread 1107 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 207146 thread 1108 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 207147 thread 1109 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 207148 thread 1110 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 207150 thread 1112 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 207149 thread 1111 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 207151 thread 1113 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 198864 tid 207152 thread 1114 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 198

In [8]:
!python preprocessing-11132019.py -c pp_config/2021_09_03_Test_for_Nando_Cl_Inv.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
09/02/2021 04:38:53 PM Preprocess training dataset with real_geography flag set to False
09/02/2021 04:38:53 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_09_03_TEST_For_Nando_ClInv.nc
09/02/2021 04:38:53 PM Reading input files
09/02/2021 04:38:53 PM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0000-0[6-9]-*-*.nc
09/02/2021 04:42:04 PM Crop levels
09/02/2021 04:42:04 PM Create stacked dataarray
09/02/2021 04:42:04 PM Real geography flag set to False
09/02/2021 04:42:04 PM Aquaplanet simulation detected
var is  RH
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-47
OMP: Info #156: KMP_AFFINITY: 48 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 2 packages x 12 co

OMP: Info #250: KMP_AFFINITY: pid 265168 tid 267048 thread 48 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268101 thread 49 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268102 thread 50 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268103 thread 51 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268104 thread 52 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268105 thread 53 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268106 thread 54 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268107 thread 55 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268108 thread 56 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268109 thread 57 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268110 thread 58 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268111 thread 

OMP: Info #250: KMP_AFFINITY: pid 265168 tid 267046 thread 144 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268206 thread 147 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268204 thread 145 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268205 thread 146 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268207 thread 148 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268208 thread 149 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268209 thread 150 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268211 thread 152 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268213 thread 154 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268210 thread 151 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268212 thread 153 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268

OMP: Info #250: KMP_AFFINITY: pid 265168 tid 267064 thread 240 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268344 thread 241 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268345 thread 242 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268346 thread 243 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268347 thread 244 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268348 thread 245 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268349 thread 246 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268350 thread 247 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268351 thread 248 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268353 thread 250 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268352 thread 249 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268

OMP: Info #250: KMP_AFFINITY: pid 265168 tid 267084 thread 336 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268447 thread 338 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268446 thread 337 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268448 thread 339 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268449 thread 340 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268450 thread 341 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268451 thread 342 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268452 thread 343 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268453 thread 344 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268454 thread 345 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268456 thread 347 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 268

...

OMP: Info #250: KMP_AFFINITY: pid 265168 tid 267041 thread 720 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269442 thread 721 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269443 thread 722 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269444 thread 723 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269445 thread 724 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269448 thread 727 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269447 thread 726 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269446 thread 725 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269449 thread 728 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269451 thread 730 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269450 thread 729 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269

OMP: Info #250: KMP_AFFINITY: pid 265168 tid 267057 thread 816 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269590 thread 817 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269591 thread 818 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269592 thread 819 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269593 thread 820 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269594 thread 821 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269595 thread 822 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269597 thread 824 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269596 thread 823 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269598 thread 825 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269599 thread 826 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269

OMP: Info #250: KMP_AFFINITY: pid 265168 tid 267059 thread 912 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269828 thread 913 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269829 thread 914 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269830 thread 915 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269831 thread 916 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269832 thread 917 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269833 thread 918 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269834 thread 919 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269835 thread 920 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269836 thread 921 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269837 thread 922 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269

OMP: Info #250: KMP_AFFINITY: pid 265168 tid 267062 thread 1008 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269992 thread 1010 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269993 thread 1011 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269994 thread 1012 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269991 thread 1009 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269995 thread 1013 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269996 thread 1014 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269998 thread 1016 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 269997 thread 1015 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270000 thread 1018 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270001 thread 1019 bound to OS proc set 14
OMP: Info #250: KMP_AFFINITY: pid 265

OMP: Info #250: KMP_AFFINITY: pid 265168 tid 267060 thread 1104 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270280 thread 1105 bound to OS proc set 4
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270282 thread 1107 bound to OS proc set 10
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270281 thread 1106 bound to OS proc set 8
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270283 thread 1108 bound to OS proc set 6
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270284 thread 1109 bound to OS proc set 2
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270285 thread 1110 bound to OS proc set 12
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270286 thread 1111 bound to OS proc set 16
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270287 thread 1112 bound to OS proc set 20
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270289 thread 1114 bound to OS proc set 18
OMP: Info #250: KMP_AFFINITY: pid 265168 tid 270288 thread 1113 bound to OS proc set 22
OMP: Info #250: KMP_AFFINITY: pid 265

In [7]:
!python preprocessing-11132019.py -c pp_config/2021_06_16_T_BMSE_input_small_data_test.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
06/16/2021 11:53:48 AM Preprocess training dataset with real_geography flag set to False
06/16/2021 11:53:48 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_06_16_BMSE_small.nc
06/16/2021 11:53:48 AM Reading input files
06/16/2021 11:53:48 AM Reading input file /DFS-L/DATA/pritchard/tbeucler/SPCAM/fluxbypass_aqua/AndKua_aqua_SPCAM3.0_sp_fbp_f4.cam2.h1.0001-0[1-2]-01-00000.nc
/nfspool-0/home/tbeucler/CBRAIN-CAM/cbrain/preprocessing/convert_dataset_20191113.py:772: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  ds = xr.open_mfdataset(in_fns, decode_times=False, decode_cf=False, concat_dim='time')
/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/pytho

In [3]:
!python preprocessing-11132019.py -c pp_config/2021_06_06_RG_Test-0K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
06/06/2021 10:28:44 AM Preprocess training dataset with real_geography flag set to True
06/06/2021 10:28:44 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_06_06_RG_TEST.nc
06/06/2021 10:28:44 AM Reading input files
06/06/2021 10:28:44 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_0001Kp_Neuralnet_SPCAM_all_216/atm/hist/SST_0001Kp_Neuralnet_SPCAM_all_216.cam.h1.2014-*-2*-*.nc
06/06/2021 10:37:38 AM Crop levels
06/06/2021 10:37:38 AM Create stacked dataarray
06/06/2021 10:37:38 AM Real geography flag set to True
06/06/2021 10:37:38 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINIT

In [3]:
!python preprocessing-11132019.py -c pp_config/2021_06_03_RG_Train_Valid-0K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
06/04/2021 07:55:18 AM Preprocess training dataset with real_geography flag set to True
06/04/2021 07:55:18 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_06_03_RG_TRAIN.nc
06/04/2021 07:55:18 AM Reading input files
06/04/2021 07:55:18 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_0001Kp_Neuralnet_SPCAM_all_216/atm/hist/SST_0001Kp_Neuralnet_SPCAM_all_216.cam.h1.2013-*-0*-*.nc
06/04/2021 08:03:40 AM Crop levels
06/04/2021 08:03:40 AM Create stacked dataarray
06/04/2021 08:03:40 AM Real geography flag set to True
06/04/2021 08:03:40 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINI

In [18]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Train_Valid-M4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/25/2021 01:07:56 AM Preprocess training dataset including output quantiles
04/25/2021 01:07:56 AM with real_geography flag set to True
04/25/2021 01:07:56 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TRAIN_M4K.nc
04/25/2021 01:07:56 AM Reading input files
04/25/2021 01:07:56 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Km_Neuralnet_SPCAM_all_small_216/atm/hist/SST_4Km_Neuralnet_SPCAM_all_small_216.cam.h1.2009-*-0*-*.nc
04/25/2021 01:12:31 AM Reading PKL file containing univariate distribution of output variables
04/25/2021 01:12:31 AM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TRAIN_M4K_shuffle.nc_PERC.pkl
04/25/2021 01:12:31 AM Crop levels
04/25/2021 01:12:31 AM Create stacked dataarray
04/25/2021 01:12:31 AM Real geography flag set to True
04/25/2021 01:12:31 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLI

OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128624 thread 24 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128626 thread 25 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128589 thread 26 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128598 thread 27 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128616 thread 28 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128617 thread 29 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128584 thread 30 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128592 thread 31 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128591 thread 32 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128601 thread 33 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128599 thread 34 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128611 thread 35 bou

04/25/2021 08:10:34 AM Done!
04/25/2021 08:10:42 AM Finish entire preprocessing script.


In [19]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Train_Valid-P4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/25/2021 08:16:12 AM Preprocess training dataset including output quantiles
04/25/2021 08:16:12 AM with real_geography flag set to True
04/25/2021 08:16:12 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TRAIN_P4K.nc
04/25/2021 08:16:12 AM Reading input files
04/25/2021 08:16:12 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Kp_Neuralnet_SPCAM_all_v2_216/atm/hist/SST_4Kp_Neuralnet_SPCAM_all_v2_216.cam.h1.2009-*-0*-*.nc
04/25/2021 08:21:25 AM Reading PKL file containing univariate distribution of output variables
04/25/2021 08:21:25 AM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TRAIN_P4K_shuffle.nc_PERC.pkl
04/25/2021 08:21:25 AM Crop levels
04/25/2021 08:21:25 AM Create stacked dataarray
04/25/2021 08:21:25 AM Real geography flag set to True
04/25/2021 08:21:25 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var 

04/25/2021 04:21:17 PM Done!
04/25/2021 04:21:22 PM Finish entire preprocessing script.


In [20]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Test-P4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/25/2021 04:26:51 PM Preprocess training dataset including output quantiles
04/25/2021 04:26:51 PM with real_geography flag set to True
04/25/2021 04:26:51 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TEST_P4K.nc
04/25/2021 04:26:51 PM Reading input files
04/25/2021 04:26:51 PM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Kp_Neuralnet_SPCAM_all_v2_216/atm/hist/SST_4Kp_Neuralnet_SPCAM_all_v2_216.cam.h1.2010-*-2*-*.nc
04/25/2021 04:32:30 PM Reading PKL file containing univariate distribution of output variables
04/25/2021 04:32:30 PM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TEST_P4K.nc_PERC.pkl
04/25/2021 04:32:30 PM Crop levels
04/25/2021 04:32:30 PM Create stacked dataarray
04/25/2021 04:32:30 PM Real geography flag set to True
04/25/2021 04:32:30 PM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX


OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101687 thread 24 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101683 thread 25 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101660 thread 26 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101693 thread 27 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101662 thread 28 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101686 thread 29 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101653 thread 30 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101685 thread 31 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101690 thread 32 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101692 thread 33 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101672 thread 34 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101655 thread 35 bound to OS pro

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Test-M4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/26/2021 04:16:59 PM Preprocess training dataset including output quantiles
04/26/2021 04:16:59 PM with real_geography flag set to True
04/26/2021 04:16:59 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TEST_M4K.nc
04/26/2021 04:16:59 PM Reading input files
04/26/2021 04:16:59 PM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Km_Neuralnet_SPCAM_all_small_216/atm/hist/SST_4Km_Neuralnet_SPCAM_all_small_216.cam.h1.2010-*-2*-*.nc
04/26/2021 04:21:47 PM Reading PKL file containing univariate distribution of output variables
04/26/2021 04:21:47 PM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TEST_M4K.nc_PERC.pkl
04/26/2021 04:21:47 PM Crop levels
04/26/2021 04:21:47 PM Create stacked dataarray
04/26/2021 04:21:47 PM Real geography flag set to True
04/26/2021 04:21:47 PM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_31_O3_T_NSto220_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_02_01_O3_BCONS_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_02_01_O3_QSATdeficit_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_input_small_data_test

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Train_Valid.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/147_POG_RHinput_FLUXoutput_test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/118_POG_TfromTSinput_test.yml

In [ ]:
!ls -la -h /local/Tom.Beucler/SPCAM_PHYS/1*

# 2) Check the preprocessed data

In [8]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

In [8]:
path_RGsmall = path_data + '2021_04_18_NORM_RG_small.nc'

In [9]:
path_QSATdeficit = path_data+'2021_02_01_NORM_O3_QSATdeficit_small.nc'

In [10]:
path_TfromNS = path_data+'2021_02_01_NORM_O3_TfromNS_small.nc'

In [9]:
path_Bcons = path_data+'2021_03_17_NORM_O3_BCONS_small.nc'

In [12]:
path_BMSE = path_data + '2021_06_16_NORM_BMSE_small.nc'

In [13]:
data_RGsmall = xr.open_dataset(path_RGsmall)

In [13]:
# data_QSATdeficit = xr.open_dataset(path_QSATdeficit)
# data_TfromNS = xr.open_dataset(path_TfromNS)
# data_Bcons = xr.open_dataset(path_Bcons)
data_Bmse = xr.open_dataset(path_BMSE)

In [ ]:
data_QSATdeficit

In [ ]:
data_TfromNS

In [14]:
data_Bmse['var_names']

<xarray.DataArray 'var_names' (var_names: 184)>
array(['QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP',
       'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP',
       'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP',
       'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE',
       'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE',
       'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE', 'BMSE',
       'BMSE', 'BMSE', 'BMSE', 'PS', 'SOLIN', 'SHFLX', 'LHFLX', 'PHQ', 'PHQ',
       'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ',
       'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ',
       'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'QRL',
       'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL',
       'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL',
       'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRL', 'QRS',
       'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS',
       'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS',
       'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS', 'QRS'],
      dtype=object)
Coordinates:
  * var_names  (var_names) object 'QBP' 'QBP' 'QBP' 'QBP' ... 'QRS' 'QRS' 'QRS'

In [ ]:
data_TfromNS['mean'].shape

In [16]:
data_Bmse['mean'][:30].values

array([1.34937473e-06, 1.31343331e-06, 1.08532610e-06, 1.15688249e-06,
       1.13117293e-06, 1.11655880e-06, 1.13724015e-06, 1.21084406e-06,
       1.37536440e-06, 1.82399217e-06, 3.32074467e-06, 7.77086272e-06,
       1.85950039e-05, 4.11459191e-05, 8.63846857e-05, 1.69385693e-04,
       3.12721841e-04, 5.58001289e-04, 9.55813639e-04, 1.53020486e-03,
       2.36349273e-03, 3.60855338e-03, 4.97421819e-03, 5.76664708e-03,
       6.34710349e-03, 6.87083335e-03, 7.30065596e-03, 7.62099791e-03,
       7.80885332e-03, 8.26899336e-03])

In [17]:
data_Bmse['mean'][30:60].values

array([-4.00554868e+00, -5.51612582e+00, -7.53976408e+00, -4.92733378e+00,
       -4.75926980e+00, -4.06667155e+00, -3.29265373e+00, -2.76421626e+00,
       -2.39479296e+00, -2.03422430e+00, -1.67829473e+00, -1.32668592e+00,
       -1.00346602e+00, -7.32355679e-01, -5.21988785e-01, -3.62944323e-01,
       -2.42846319e-01, -1.49522538e-01, -7.19474030e-02, -7.33157626e-03,
        3.52665651e-02,  5.46795283e-02,  5.61516069e-02,  5.10403021e-02,
        4.21202158e-02,  3.24117022e-02,  2.38070651e-02,  1.66014297e-02,
        1.35225718e-02, -2.98660228e-04])

In [ ]:
plt.hist(data_Bmse['mean'])

In [ ]:
data_Bcons['mean'][30:60]

In [ ]:
data_Bcons['std'][30:60]

# 3) Preprocessing for PRL paper

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-RH-test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-test.yml